In [1]:
import torch
import torch.nn as nn

In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [3]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 636 kB/s 
     |████████████████████████████████| 138 kB 16.2 MB/s 
     |████████████████████████████████| 178 kB 37.9 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 145 kB 61.2 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=2fd277981a55b0e6f385644dfed60001bd22bffeda5b62f380a6b553a6b939f9
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certi

In [4]:
import roboflow   # may need to !pip install roboflow into your environment first, followed from: https://pypi.org/project/roboflow/#:~:text=import%20roboflow%20%23%20Instantiate%20Roboflow%20object%20with%20your,%23%20Save%20the%20prediction%20as%20an%20image%20prediction.save%28output_path%3D%27predictions.jpg%27%29

In [5]:
rf = roboflow.Roboflow(api_key='DXb4B94A1xpmpoSfBs45') #private API key from Roboflow

# Roboflow-cli

In [20]:
#@title Default title text
# followed this tutorial to install command line interface: https://docs.roboflow.com/command-line-interface#using-the-roboflow-cli

!npm i -g roboflow-cli

/tools/node/bin/roboflow -> /tools/node/lib/node_modules/roboflow-cli/cli/index.js
+ roboflow-cli@0.0.16
added 347 packages from 201 contributors in 14.595s


In [21]:
!roboflow auth

opening webrowser for you to log in and retrieve auth token...

****************
if you are on a headless system without a browser, open the following url in another browser to retrieve your auth token:

https://app.roboflow.com/auth-cli

****************


? Please paste the cli auth token displayed in the webrowser ‣  ✔ Please paste the cli auth token displayed in the webrowser · f216a566-f881-46e7-88da-32e7d38c3ebe✔ Please paste the cli auth token displayed in the webrowser · f216a566-f881-46e7-88da-32e7d38c3ebe✔ Please paste the cli auth token displayed in the webrowser · f216a566-f881-46e7-88da-32e7d38c3ebe✔ Please paste the cli auth token displayed in the webrowser · f216a566-f881-46e7-88da-32e7d38c3ebe✔ Please paste the cli auth token displayed in the webrowser · f216a566-f881-46e7-88da-32e7d38c3ebe✔ Please paste the cli auth token displayed in the webrowser · f216a566-f881-46e7-88da-32e7d38c3ebe✔ Please paste the cli auth token displayed in the webrowser · f216a566-f881-46e7-88

In [24]:
!roboflow project list
!roboflow project get itml-project/itml-final-project-ai-versus-non-ai-dataset


 Hard Hat Sample
  link: https://app.roboflow.com/itml-project/hard-hat-sample-tudx0
  id: itml-project/hard-hat-sample-tudx0
  type: object-detection
  versions: 2
  images: 100
  classes: head,helmet,person

 ITML Final Project AI versus Non AI Dataset
  link: https://app.roboflow.com/itml-project/itml-final-project-ai-versus-non-ai-dataset
  id: itml-project/itml-final-project-ai-versus-non-ai-dataset
  type: classification
  versions: 1
  images: 1032
  classes: AI,nonai

itml-project itml-final-project-ai-versus-non-ai-dataset
{
  workspace: { name: 'ITML Project', url: 'itml-project', members: 3 },
  project: {
    id: 'itml-project/itml-final-project-ai-versus-non-ai-dataset',
    type: 'classification',
    name: 'ITML Final Project AI versus Non AI Dataset',
    created: 1670357839.567,
    updated: 1670528974.226,
    images: 1032,
    unannotated: 0,
    annotation: 'Image-Classification',
    versions: 1,
    public: true,
    license: 'CC BY 4.0',
    splits: { export: 0,

# Dataset

In [6]:
workspace = rf.workspace()
project = rf.project("itml-project/itml-final-project-ai-versus-non-ai-dataset") #ID paste here

loading Roboflow workspace...


In [9]:
!roboflow project get "itml-project/itml-final-project-ai-versus-non-ai-dataset"

/bin/bash: roboflow: command not found


In [9]:
dataset = project.version(1).download("tfrecord")

Extracting Dataset Version Zip to ITML-Final-Project-AI-versus-Non-AI-Dataset-1 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 12.31it/s]


In [10]:
type(dataset)

roboflow.core.dataset.Dataset

In [13]:
import tensorflow as tf
path = '/content/ITML-Final-Project-AI-versus-Non-AI-Dataset-1/test/Image-Classification.tfrecord'
raw_data = tf.data.TFRecordDataset(path)


In [18]:
raw_image_dataset = tf.data.TFRecordDataset(path)

# Create a dictionary describing the features.
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset element_spec={'depth': TensorSpec(shape=(), dtype=tf.int64, name=None), 'height': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image_raw': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'width': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [19]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))

InvalidArgumentError: ignored

In [17]:
import torchvision
import imageio
import os

In [43]:
data_dir_train = '/content/ITML/train/images'
data_dir_test = '/content/ITML/test/images'

label_dir_train = '/content/ITML/train/labels'
label_dir_test = '/content/ITML/test/labels'

files_train = [name for name in os.listdir(data_dir_train)]
files_test = [name for name in os.listdir(data_dir_test)]

i=0
for filename in os.listdir(label_dir_train):
  filename = label_dir_train + '/' + filename
  f = open(filename, 'r')
  labels_train[i] = float(f.readline())
  filename.close()
  i +=1
  
i=0
for filename in os.listdir(label_dir_test):
  filename = label_dir_test + '/' + filename
  f = open(filename, 'r')
  labels_test[i] = float(f.readline())
  f.close()
  i += 1

labels_train = [float(label) for label in os.listdir(label_dir_train)]
labels_test = []

batch_size_train = 721
batch_size_test = 106

batch_train = torch.zeros(batch_size_train, 3, 512, 512, dtype = torch.uint8)
batch_test = torch.zeros(batch_size_test, 3, 512, 512, dtype = torch.uint8)

ValueError: ignored

In [30]:
for i, filename in enumerate(files_train):
  img_arr = imageio.imread(os.path.join(data_dir_train, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  img_t = img_t[:3]
  batch_train[i] = (img_t, 1)

for i, filename in enumerate(files_test):
  img_arr = imageio.imread(os.path.join(data_dir_test, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  img_t = img_t[:3]
  batch_test[i] = (img_t, label)

In [32]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [33]:
class data_set_train(Dataset):
    def __init__(self):
        self.data = batch_train
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, index):
        return self.data[index]

In [34]:
class data_set_test(Dataset):
    def __init__(self):
        self.data = batch_test
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, index):
        return self.data[index]

In [ ]:
dataloader_train = DataLoader(data_set_train, batch_size = 10, shuffle = False)
